In [1]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
from ultralytics import YOLO

In [3]:
# model = YOLO("yolo11l-pose.pt")
# model.export(format="engine")

In [4]:
!mv yolo11l-pose.pt ../data/models
!mv yolo11l-pose.onnx ../data/models

'mv' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
ENGINE_PATH = "../data/models/yolo11l-pose.engine"
VIDEO_INPUT = "../data/3196221-uhd_3840_2160_25fps.mp4"
VIDEO_OUTPUT = "../data/yolo11l_pose_tracked.mp4"
CSV_OUTPUT = "../data/yolo11l_pose_tracked.csv"

In [6]:
POSE_CONNECTIONS = [
    (5, 7), (7, 9),      # left arm
    (6, 8), (8, 10),     # right arm
    (5, 6),              # shoulders
    (5, 11), (6, 12),    # torso
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16)   # right leg
]

In [7]:
model = YOLO(ENGINE_PATH)

In [8]:
def draw_pose(frame, kpts, track_id):
    h, w, _ = frame.shape

    for x, y, conf in kpts:
        if conf > 0.3:
            cv2.circle(frame, (int(x), int(y)), 4, (0, 0, 255), -1)

    for a, b in POSE_CONNECTIONS:
        if kpts[a][2] > 0.3 and kpts[b][2] > 0.3:
            x1, y1 = int(kpts[a][0]), int(kpts[a][1])
            x2, y2 = int(kpts[b][0]), int(kpts[b][1])
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if track_id is not None:
        cx, cy = int(kpts[0][0]), int(kpts[0][1])
        cv2.putText(
            frame, f"ID {track_id}",
            (cx, cy - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6, (255, 255, 0), 2
        )


In [9]:
cap = cv2.VideoCapture(VIDEO_INPUT)

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, fps, (w, h))


In [10]:
records = []

with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc="Tracking") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(
            frame,
            persist=True,
            conf=0.4,
            iou=0.5,
            tracker="bytetrack.yaml",
            verbose=False
        )

        if results and results[0].keypoints is not None:
            boxes = results[0].boxes
            kpts = results[0].keypoints.xy.cpu().numpy()
            confs = results[0].keypoints.conf.cpu().numpy()

            track_ids = (
                boxes.id.cpu().numpy().astype(int)
                if boxes.id is not None else [None] * len(kpts)
            )

            for i, (kp, kp_conf) in enumerate(zip(kpts, confs)):
                person_id = track_ids[i]
                kpts_full = np.hstack([kp, kp_conf[:, None]])

                draw_pose(frame, kpts_full, person_id)

                row = {
                    "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
                    "track_id": person_id
                }

                for j, (x, y, c) in enumerate(kpts_full):
                    row[f"kp{j}_x"] = x
                    row[f"kp{j}_y"] = y
                    row[f"kp{j}_conf"] = c

                records.append(row)
        print(type(results[0].keypoints), results[0].keypoints is None)
        out.write(frame)
        pbar.update(1)

cap.release()
out.release()


Tracking:   0%|          | 0/385 [00:00<?, ?it/s]

Loading ../data/models/yolo11l-pose.engine for TensorRT inference...


Tracking:   1%|          | 3/385 [00:06<10:36,  1.67s/it]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking:   1%|▏         | 5/385 [00:06<05:21,  1.18it/s]

...

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking:  99%|█████████▉| 383/385 [00:36<00:00, 11.97it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking: 100%|██████████| 385/385 [00:36<00:00, 10.55it/s]

<class 'ultralytics.engine.results.Keypoints'> False


In [11]:
df = pd.DataFrame(records)
df.to_csv(CSV_OUTPUT, index=False)

print("Done.")
print("Video:", VIDEO_OUTPUT)
print("CSV:", CSV_OUTPUT)


Done.
Video: ../data/yolo11l_pose_tracked.mp4
CSV: ../data/yolo11l_pose_tracked.csv
